# <center>Similarity Test</center>

In [1]:
# Imports
import glob
from music21 import converter, instrument, note, chord, stream, duration
from fuzzywuzzy import fuzz
import numpy as np

C:\Users\Michael\Anaconda3\envs\CITS4404\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


First get all of the notes for the songs to compare the input to

In [2]:
def get_notes(path):
    """
        Gets all notes and chords from midi file
    """
    notes = []
    names = []

    for file in glob.glob(path + "*.mid"):        
        song = []
        midi = converter.parse(file)
        
        # Get some info about the file
        key = midi.analyze('key')
        key_string = key.tonic.name + ' ' + key.mode
        
        print("Parsing %s" % file)
        print(key_string)
        
        notes_to_parse = None

        try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse() 
            print(len(s2.parts))
        except: # file has notes in a flat structure
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                song.append([str(element.pitch), element.offset, element.duration])
            elif isinstance(element, chord.Chord):
                song_note = '.'.join(str(n) for n in element.normalOrder)
                song.append([song_note, element.offset, element.duration])
        notes.append(np.array(song))
        names.append(file.strip(path + '\\'))

    return notes, names

comp_path = "../Similarity/compare_to/"
comp_notes, comp_names = get_notes(comp_path)

Parsing ../Similarity/compare_to\Pokemon Gold, Silver, Crystal - Cinnabar Island (HGSS Version).mid
G major
1
Parsing ../Similarity/compare_to\Pokemon Gold, Silver, Crystal - S.S. Aqua .mid
G major
1
Parsing ../Similarity/compare_to\Pokemon GoldSilverCrystal - Azalea TownBlackthorn City.mid
C# major
1
Parsing ../Similarity/compare_to\Pokemon GoldSilverCrystal - Bicycle.mid
E minor
1
Parsing ../Similarity/compare_to\Pokemon GoldSilverCrystal - Bug Catching Contest.mid
E minor
1
Parsing ../Similarity/compare_to\Pokemon GoldSilverCrystal - Burned Tower.mid
E minor
1
Parsing ../Similarity/compare_to\Pokemon GoldSilverCrystal - Champion Battle.mid
G# minor
1
Parsing ../Similarity/compare_to\Pokemon GoldSilverCrystal - Cherrygrove CityMahogany Town.mid
F major
1
Parsing ../Similarity/compare_to\Pokemon GoldSilverCrystal - Dance Theatre.mid
A minor
1
Parsing ../Similarity/compare_to\Pokemon GoldSilverCrystal - Dark Cave.mid
A- major
1
Parsing ../Similarity/compare_to\Pokemon GoldSilverCrystal

Process the similarities

In [3]:
sep = " "
comp_pitches = []
comp_offsets = []
comp_durations = []
for song in comp_notes:
    comp_pitches.append(sep.join(song[:,0]))
    comp_offsets.append(sep.join([str(round(float(x), 2)) for x in song[:,1]]))
    comp_durations.append(sep.join([str(round(float(x.quarterLength), 2)) for x in song[:,2]]))

Next, get the input song. I will assume this is the first song received.

In [15]:
input_path = "../Similarity/input/"
notes, names = get_notes(input_path)

Parsing ../Similarity/input\LSTM_output_piano_6.mid
D minor
1


In [16]:
input_notes = notes[0]
input_name = names[0]

I will then process the input song.

In [17]:
sep = " "
input_pitches = sep.join(input_notes[:,0])
input_offsets = sep.join([str(round(float(x), 2)) for x in input_notes[:,1]])
input_durations = sep.join([str(round(float(x.quarterLength), 2)) for x in input_notes[:,2]])

Now, I will use the fuzz library to test the similarity of songs. I will consider pitch, offset and duration as equally weighted so I will consider the similarity score the weighted average of the three.

I chose to consider similarity based on partial ratio because this considers the similarity of sentences, rather than exact words. I expect many of the elements to be copied over from the songs, so simply counting how many are used in both is too basic to compare realistically.<br>
This is the best measure for music because it allows for the same notes to be played with other notes in between without penalization in score, which is what would be desired - sounding similar, while trying new things.

I will present them in descending order.

In [18]:
similarity_scores = dict()
for (comp_pitch, comp_offset, comp_duration, name) in zip(comp_pitches, comp_offsets, comp_durations, comp_names):
    similarity_scores[name] = round((fuzz.partial_ratio(input_pitches, comp_pitch) + fuzz.partial_ratio(input_offsets, comp_offset) + fuzz.partial_ratio(input_durations, comp_duration))/3, 2)

In [19]:
import operator
sorted_similarity_scores = sorted(similarity_scores.items(), key=operator.itemgetter(1), reverse=True)
print('Similary Score (%)  -      Song Name')
for tup in sorted_similarity_scores:
    print(f'{tup[1]:.2f}                -     {tup[0]}')

Similary Score (%)  -      Song Name
10.67                -     Pokemon GoldSilverCrystal - Dance Theatre.mid
5.33                -     Pokemon GoldSilverCrystal - Champion Battle.mid
4.67                -     Pokemon GoldSilverCrystal - Sprout Tower.mid
4.33                -     Pokemon GoldSilverCrystal - Lavender Town.mid
4.00                -     Pokemon GoldSilverCrystal - Azalea TownBlackthorn City.mid
4.00                -     Pokemon GoldSilverCrystal - Gym.mid
4.00                -     Pokemon GoldSilverCrystal - Johto Wild Pokemon Battle.mid
3.67                -     Pokemon GoldSilverCrystal - Pokemon Lullaby.mid
3.33                -     Pokemon GoldSilverCrystal - Dark Cave.mid
3.33                -     Pokemon GoldSilverCrystal - Girl Trainer Confrontation.mid
3.33                -     Pokemon GoldSilverCrystal - Victory Road.mid
3.33                -     Pokemon HeartGoldSoulSilver - Cerulean City.mid
3.33                -     Pokemon HeartGoldSoulSilver - Route 4748.mid